#### Mikhail Fedorov B20-CS-01
Work is done with the help of the following sources:
- Labs from the course
- Stackoverflow
- Copilot

# 1. Crawler

## 1.0. Related example

This code shows `wget`-like tool written in python. Run it from console (`python wget.py`), make it work. Check the code, reuse, and modify for your needs.

In [ ]:
!pip install requests
!pip install beautifulsoup4

In [2]:
import argparse
import os
import re
import requests


def wget(url, filename):
    # allow redirects - in case file is relocated
    resp = requests.get(url, allow_redirects=True)
    # this can also be 2xx, but for simplicity now we stick to 200
    # you can also check for `resp.ok`
    # check for resp.ok
    if not resp.ok:
        print(resp.status_code, resp.reason, 'for', url)
        return
    
    # just to be cool and print something
    print(*[f"{key}: {value}" for key, value in resp.headers.items()], sep='\n')
    print()
    
    # try to extract filename from url
    if filename is None:
        # start with http*, ends if ? or # appears (or none of)
        m = re.search("^http.*/([^/\?#]*)[\?#]?", url)
        filename = m.group(1)
        if not filename:
            raise NameError(f"Filename neither given, nor found for {url}")

    # what will you do in case 2 websites store file with the same name?
    if os.path.exists(filename):
        raise OSError(f"File {filename} already exists")
    
    with open(filename, 'wb') as f:
        f.write(resp.content)
        print(f"File saved as {filename}")


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='download file.')
    parser.add_argument("-O", type=str, default=None, dest='filename', help="output file name. Default -- taken from resource")
    parser.add_argument("url", type=str, default=None, help="Provide URL here")
    args = parser.parse_args()
    wget(args.url, args.filename)

usage: ipykernel_launcher.py [-h] [-O FILENAME] url
ipykernel_launcher.py: error: the following arguments are required: url


SystemExit: 2

/home/max/uni/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### 1.0.1. How to parse a page?

If you build a crawler, you might follow one of the approaches:
1. search for URLs in the page, assuming this is just a text.
2. search for URLs in the places where URLs should appear: `<a href=..`, `<img src=...`, `<iframe src=...` and so on.

To follow the first approach you can rely on some good regular expression. [Like this](https://stackoverflow.com/a/3809435).

To follow the second approach just read one of these: [short answer](https://stackoverflow.com/questions/1080411/retrieve-links-from-web-page-using-python-and-beautifulsoup) or [exhaustive explanation](https://hackersandslackers.com/scraping-urls-with-beautifulsoup/).

## 1.1. [15] Download and persist #
Please complete a code for `load()`, `download()` and `persist()` methods of `Document` class. What they do:
- for a given URL `download()` method downloads binary data and stores in `self.content`. It returns `True` for success, else `False`.
- `persist()` method saves `self.content` somewhere in file system. We do it to avoid multiple downloads (for caching in other words).
- `load()` method loads data from hard drive. Returns `True` for success.

Tests checks that your code somehow works.

**NB Passing the test doesn't mean you correctly completed the task.** These are **criteria, which have to be fullfilled**:
1. URL is a unique identifier (as it is a subset of URI). Thus, documents with different URLs should be stored in different files. Typical errors: documents from the same domain are overwritten to the same file, URLs with similar endings are downloaded to the same file, etc.
2. The document can be not only a text file, but also a binary. Pay attention that if you download `mp3` file, it still can be played. Hint: don't hurry to convert everything to text.

In [7]:
import requests
from urllib.parse import quote
import os

class Document:
    
    def __init__(self, url, index=None):
        self.url = url
        self.content = None
        self.index = index
        self.get()
        
    def get(self):
        if not self.load():
            if not self.download():
                raise FileNotFoundError(self.url)
            else:
                self.persist()
    
    def download(self):
        #TODO downloads binary data from self.url and stores in `self.content`. It returns `True` for success, else `False`
        resp = requests.get(self.url, allow_redirects=True)
        if not resp.ok:
            # Can be used for debugging
            # print(resp.status_code, resp.reason, 'for', self.url)
            return False
        self.content = resp.content
        return True
    
    def persist(self):
        name = quote(self.url, safe='')
        if self.index:
            name = str(self.index) + quote(self.url, safe='')
        #TODO write document content to hard drive
        with open(name, 'wb') as f:
            f.write(self.content)
            # Can be used for debugging
            # print(f"File for {self.url} saved as {quote(self.url, safe='')} on disk")
            
    def load(self):
        #TODO load content from hard drive, store it in self.content and return True in case of success
        name = quote(self.url, safe='')
        if self.index:
            name = str(self.index) + quote(self.url, safe='')
        if not os.path.exists(name):
            # Can be used for debugging
            # print(f"File for {self.url} does not exist on disk")
            return False
        with open(name, 'rb') as f:
            self.content = f.read()
            return True

### 1.1.1. Tests ###

In [ ]:
doc = Document('http://sprotasov.ru/data/iu.txt')
# Check for mp3 file
# doc = Document('https://www.soundhelix.com/examples/mp3/SoundHelix-Song-1.mp3')

doc.get()
assert doc.content, "Document download failed"
# check mp3 file content
# assert doc.content[:3] == b'ID3', "Document content error"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document content error"

doc.get()
assert doc.load(), "Load should return true for saved document"
# assert doc.content[:3] == b'ID3', "Document load from disk error"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document load from disk error"

In [ ]:
# links = ['https://ke-images-dev.servicecdn.ru/cd7sh4oduq1qjncapc60/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4oduq1qjncapbvg/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4gduq1qjncapbp0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4oduq1qjncapc3g/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4gduq1qjncapbo0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4gduq1qjncapbq0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4oduq1qjncapc20/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4gduq1qjncapbtg/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4gduq1qjncapbpg/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4gduq1qjncapbog/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4oduq1qjncapc00/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4gduq1qjncapblg/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4oduq1qjncapbv0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4gduq1qjncapbqg/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4oduq1qjncapc40/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4oduq1qjncapc2g/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4oduq1qjncapc50/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4gduq1qjncapbm0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4gduq1qjncapbng/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4gduq1qjncapbrg/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4gduq1qjncapbt0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4gduq1qjncapbmg/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4oduq1qjncapc10/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4gduq1qjncapbn0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4oduq1qjncapc5g/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4gduq1qjncapbu0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4oduq1qjncapc6g/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4oduq1qjncapc1g/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4gduq1qjncapbs0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4oduq1qjncapc70/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4oduq1qjncapc4g/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4oduq1qjncapc0g/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4gduq1qjncapbsg/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4oduq1qjncapc30/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4gduq1qjncapbug/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7sh4gduq1qjncapbr0/360_720x960.jpg']

# docs = [Document(link) for link in links]
# [doc.persist() for doc in docs]

## 1.2. [10] Parse HTML
`BeautifulSoap` library is a de facto standard to parse XML and HTML documents in python. Use it to complete `parse()` method that extracts document contents. You should initialize:
1. `self.anchors` list of tuples `('text', 'url')` met in a document. Be aware, there exist relative links (e.g. `../content/pic.jpg`). Use `urllib.parse.urljoin()` to fix this issue.
2. `self.images` list of images met in a document. Again, links can be relative to current page.
3. `self.text` should keep plain text of the document without scripts, tags, comments and so on. You can refer to [this stackoverflow answer](https://stackoverflow.com/a/1983219) for details.

**NB All these 3 criteria must be fulfilled to get full point for the task.**

In [8]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.parse


class HtmlDocument(Document):
    def __init__(self, url, index=None):
        super().__init__(url, index)
        self.url = url
        self.anchors = []
        self.images = []
        self.text = ""

    def tag_visible(self, element):
        if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
            return False
        if isinstance(element, Comment):
            return False
        return True


    def text_from_html(self):
        soup = BeautifulSoup(self.content, 'html.parser')
        texts = soup.findAll(string=True)
        visible_texts = filter(self.tag_visible, texts)  
        return u" ".join(t.strip() for t in visible_texts)

    def extract_images(self):
        #TODO extract images from the document
        soup = BeautifulSoup(self.content, 'html.parser')
        images = soup.findAll('img')
        for image in images:
            self.images.append(urllib.parse.urljoin(self.url, image['src']))
        return self.images

    def extract_anchors(self):
        # TODO extract links from the document in the form of (text, url)
        soup = BeautifulSoup(self.content, 'html.parser')
        anchors = soup.findAll('a')
        for anchor in anchors:
            try:
                self.anchors.append((anchor.text, urllib.parse.urljoin(self.url, anchor['href'])))
            except KeyError:
                pass
        return self.anchors

    
    def parse(self):
        #TODO extract plain text, images and links from the document
        self.anchors = self.extract_anchors()
        self.images = self.extract_images()
        self.text = self.text_from_html()

### 1.2.1. Tests

In [ ]:
doc = HtmlDocument("http://sprotasov.ru")
doc.get()
doc.parse()

assert "just few links" in doc.text, "Error parsing text"
assert "http://sprotasov.ru/images/gb.svg" in doc.images, "Error parsing images"
assert any(p[1] == "https://twitter.com/07C3" for p in doc.anchors), "Error parsing links"

In [9]:
links = ['https://ke-images-dev.servicecdn.ru/cd7rp2oduq1qjncapa60/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp2oduq1qjncapa6g/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp2oduq1qjncapa70/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp2oduq1qjncapa7g/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp2oduq1qjncapa80/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp2oduq1qjncapa8g/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp2oduq1qjncapa90/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp30duq1qjncapa9g/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp30duq1qjncapaa0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp30duq1qjncapaag/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp30duq1qjncapab0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp30duq1qjncapabg/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp30duq1qjncapac0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp30duq1qjncapacg/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp30duq1qjncapad0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp30duq1qjncapadg/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp30duq1qjncapae0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp30duq1qjncapaeg/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp30duq1qjncapaf0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp30duq1qjncapafg/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp30duq1qjncapag0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp30duq1qjncapagg/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp30duq1qjncapah0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp30duq1qjncapahg/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp30duq1qjncapai0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp30duq1qjncapaig/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp38duq1qjncapaj0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp38duq1qjncapajg/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp38duq1qjncapak0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp38duq1qjncapakg/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp38duq1qjncapal0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp38duq1qjncapalg/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp38duq1qjncapam0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp38duq1qjncapamg/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp38duq1qjncapan0/360_720x960.jpg', 'https://ke-images-dev.servicecdn.ru/cd7rp38duq1qjncapang/360_720x960.jpg']

images = [HtmlDocument(link, index) for index, link in enumerate(links)]
# print([image.content for image in images])


## 1.3. [10] Document analysis ##
Complete the code for `HtmlDocumentTextData` class. Implement word and sentence splitting (use any method you can propose). 

**Criteria to succeed in the task**: 
1. Your `get_word_stats()` method should return `Counter` object.
2. Don't forget to lowercase your words for counting.
3. Sentences should be obtained from inside `<body>` tag only.

In [ ]:
from collections import Counter
import nltk

class HtmlDocumentTextData:
    
    def __init__(self, url):
        self.doc = HtmlDocument(url)
        self.doc.get()
        self.doc.parse()
    
    def get_sentences(self):
        #TODO implement sentence parser
        soup = BeautifulSoup(self.doc.content, 'html.parser')
        body = soup.find('body')
        sentences = nltk.sent_tokenize(body.text, language='russian')
        return sentences

    def get_word_stats(self):
        #TODO return Counter object of the document, containing mapping {`word` -> count_in_doc}
        # Don't forget to lowercase your words for counting using nltk.word_tokenize
        text = self.doc.text
        # in lower case
        words = nltk.word_tokenize(text.lower(), language='russian')
        # remove punctuation
        words = [word for word in words if word.isalpha()]
        # convert to lowercase
        return Counter(words)

### 1.3.1. Tests ###

In [ ]:
doc = HtmlDocumentTextData("https://innopolis.university/")

print(doc.get_word_stats().most_common(10))
assert [x for x in doc.get_word_stats().most_common(10) if x[0] == 'иннополис'], 'иннополис should be among most common'

## 1.4. [15] Crawling ##

Method `crawl_generator()` is given starting url (`source`) and max depth of search. It should return a **generator** of `HtmlDocumentTextData` objects (return a document as soon as it is downloaded and parsed). You can benefit from `yield obj_name` python construction. Use `HtmlDocumentTextData.anchors` field to go deeper.

In [ ]:
from queue import Queue

class Crawler:
    
    def crawl_generator(self, source, depth=1):
        #TODO return real crawling results. Don't forget to process failures, 
        # exceptions, 3**, 4** codes
        q = Queue()
        q.put([source, 1])
        visited = set()
        while not q.empty():
            url, d = q.get()
            if url in visited or d > depth:
                continue
            visited.add(url)
            try:
                doc_data = HtmlDocumentTextData(url)
                yield doc_data
                if doc_data.doc.anchors:
                    for anchor in doc_data.doc.anchors:
                        q.put([anchor[1], d + 1])
            except FileNotFoundError:
                print(f"File for {url} does not exist on disk")
            except Exception as e:
                print(f"Error {e} for {url}")

### 1.4.1. Tests ###

In [ ]:
crawler = Crawler()
counter = Counter()

for c in crawler.crawl_generator("https://innopolis.university/en/", 2):
    print(c.doc.url)
    if c.doc.url[-4:] in ('.pdf', '.mp3', '.avi', '.mp4', '.txt'):
        print("Skipping", c.doc.url)
        continue
    counter.update(c.get_word_stats())
    print(len(counter), "distinct word(s) so far")
    
print("Done")

print(counter.most_common(20))
assert [x for x in counter.most_common(20) if x[0] == 'innopolis'], 'innopolis sould be among most common'